In [1]:
from multiprocessing import Pool
from subprocess import call
import re
import datetime as dt
from datetime import datetime
import time
import random
import json
import sys
import requests
import os
import wget
import pickle


In [8]:
root ="/datasets/sagarj/vine2016/Dataset/"
postData = "savedPosts/"
userData = "Users/"
videoData = "Videos/"
meta = "Meta/"

visitedPosts = root + "2016visitedPosts.pk"
visitedUsers = root + "2016visitedUsers.pk"

In [9]:
def getList(listFile):
    visited = []
    try:
        f = open(listFile, 'rb')
        visited = pickle.load(f)
        f.close()
    except IOError:
        f = open(listFile,"a+")
        pickle.dump([],f)
        f.close()
    return visited

def updateList(listFile, listObj):
    with open(listFile, 'wb') as f:
        pickle.dump(listObj, f)
        
def getJson(jsonFile):
    f = open(jsonFile ,'r')
    data = json.load(f)
    return data

def getPostMeta(postID , postDir):
    postData = requests.get("https://api.vineapp.com/timelines/posts/"+str(postID))
    post = open( postDir + "/" + str(postID) + '.json', 'w')
    json.dump(postData.json(), post)

def getUserData(userID , userDir ):
    userData = requests.get("https://api.vineapp.com/timelines/users/"+str(userID)+"?size=50");
    user = open( userDir + "/" + str(userID) + '.json', 'w')
    json.dump(userData.json(), user)
    
def getVideo(post , videoDir):
    videoString = post['videoUrl']
    if videoString:
        argsVideo = ['wget', '-r', '-l', '1', '-p', '-P' , videoDir, videoString ]
        call(argsVideo)
        
def now_time():
    now=dt.datetime.now()
    return int(time.mktime(now.timetuple()))

def callCmd(args):
    call(args)

In [10]:
postDir = root + postData
videoDir = root + videoData
userDir = root + userData
metaDir = root + meta

runTime = str(now_time());

for i in range(1,2):
    channelRecent = requests.get("https://vine.co/api/timelines/channels/" + str(i) + "/recent" + "?size=100")
    f = open(metaDir + runTime + "_" + str(i) +'.json', 'w')
    channelJson = channelRecent.json()
    
    if not isinstance(channelJson['data'],dict):
        print "Fault: JSON Data not a dict"
        break

    json.dump(channelJson, f)
    data = channelJson['data']
    records = data['records']

    for i in range (0 , len(records)):
        subRecord = records[i]
        crawledPosts = getList(visitedPosts)
        postid = subRecord['postId']
        
        if postid in crawledPosts:
            break
        else:
            getUserData(subRecord['userId'],userData)
            getPostMeta(subRecord['postId'],postdata)
            getVideo(subRecord, videoData)                      
        
    time.sleep(30)

print "Done Crawling 18 channels for new videos , Sleeping now"

IOError: [Errno 2] No such file or directory: 'Users//1014722016034594816.json'